In [ ]:
!pip install deep-translator
!pip install pandas langdetect

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import re
from deep_translator import GoogleTranslator
from langdetect import detect

BASE_URL  = "https://repositorio.upch.edu.pe"
START_URL = f"{BASE_URL}/handle/20.500.12866/1318/recent-submissions"
#MAX_ARTICULOS = 20  # Cambia si deseas más artículos
MAX_ARTICULOS = float('inf')
datos = []

def limpia_texto(t):
    return re.sub(r'\s+', ' ', t).strip()

def traducir(texto):
    if texto.strip() == "" or texto == "NA":
        return "NA"
    try:
        return GoogleTranslator(source='auto', target='en').translate(texto)
    except Exception as e:
        print(f"Error al traducir: {e}")
        return "NA"

def traducir_si_espanol(texto):
    if texto.strip() == "" or texto == "NA":
        return "NA"
    try:
        if detect(texto) == 'es':
            return traducir(texto)
        return texto  # Ya está en inglés u otro idioma
    except Exception as e:
        print(f"Error detectando idioma: {e}")
        return texto  # Devuelve el original si no se puede detectar

def extraer_resumen_completo(url):
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        resumen_div = soup.select_one("div.simple-item-view-description")

        if not resumen_div:
            return "NA"

        # Convertir el HTML interno a texto dividido por los spacers
        raw_html = str(resumen_div)
        partes = raw_html.split('<div class="spacer">')

        if len(partes) < 3:
            # Si no hay al menos dos separadores, devuelve todo el texto
            texto_completo = BeautifulSoup(raw_html, "html.parser").get_text(" ", strip=True)
            return limpia_texto(re.sub(r'^Resumen[:：]?\s*', '', texto_completo, flags=re.IGNORECASE))

        # La parte en inglés está entre el primer y segundo spacer => partes[1]
        texto_ingles_html = partes[1]
        texto_ingles = BeautifulSoup(texto_ingles_html, "html.parser").get_text(" ", strip=True)
        texto_ingles = limpia_texto(re.sub(r'^Resumen[:：]?\s*', '', texto_ingles, flags=re.IGNORECASE))

        return texto_ingles if texto_ingles.strip() else "NA"

    except Exception as e:
        print(f"Error al extraer resumen en {url}: {e}")
        return "NA"

def scrape_listing_page(url):
    global datos
    print(f"Scraping: {url}")
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    items = soup.select("li.ds-artifact-item")

    """
    for item in items:
        if len(datos) >= MAX_ARTICULOS:
            return
    """
    for item in items:

        titulo_tag    = item.select_one("div.artifact-title a")
        autores_tag   = item.select_one("span.author")
        fecha_tag     = item.select_one("span.date")
        publisher_tag = item.select_one("span.publisher")

        titulo    = limpia_texto(titulo_tag.text) if titulo_tag else "NA"
        link      = BASE_URL + titulo_tag["href"] if titulo_tag else "NA"
        autores   = limpia_texto(autores_tag.text) if autores_tag else "NA"
        fecha     = limpia_texto(fecha_tag.text) if fecha_tag else "NA"
        editorial = limpia_texto(publisher_tag.text) if publisher_tag else "NA"
        resumen   = extraer_resumen_completo(link) if link != "NA" else "NA"

        print(f"Traduciendo artículo {len(datos)+1}: {titulo[:50]}...")
        datos.append({
            "title": traducir_si_espanol(titulo),
            "abstract": traducir_si_espanol(resumen),
            "publisher": traducir_si_espanol(editorial),
            "authors": autores,
            "date": fecha,
            "link": link  # No se traduce
        })

    # Recorrer siguiente página si existe
    next_btn = soup.select_one("a.next-page-link")
    if next_btn and next_btn.get("href") and len(datos) < MAX_ARTICULOS:
        scrape_listing_page(BASE_URL + next_btn["href"])

# Ejecutar scraping y traducción
scrape_listing_page(START_URL)

# Guardar resultados traducidos en formato compatible con Excel en español (delimitador ;)
with open("articulos_upch_traducidos_con_editorial.csv", "w", newline="", encoding="utf-8") as f:
    fieldnames = ["title", "abstract", "publisher", "authors", "date", "link"]
    writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter=';', quoting=csv.QUOTE_ALL)
    writer.writeheader()
    writer.writerows(datos)

print(f"\n Se guardaron {len(datos)} artículos traducidos en 'articulos_upch_traducidos_con_editorial.csv'")

In [ ]:
from google.colab import files
files.download("articulos_upch_traducidos_con_editorial.csv")